2. Verdadera democracia. Suponga que usted es el jefe de gobierno y está interesado en que pasen
los proyectos de su programa político. Sin embargo, en el congreso conformado por 5 partidos, no
es fácil su tránsito, por lo que debe repartir el poder, conformado por ministerios y otras agencias
del gobierno, con base en la representación de cada partido. Cada entidad estatal tiene un  peso de
poder, que es el que se debe distribuir. Suponga que hay 50 curules, distribuya aleatoriamente, con
una distribución no informe entre los 5 partidos esas curules. Defina una lista de 50 entidades y
asígneles aleatoriamente un peso político de 1 a 100 puntos.  Cree una matriz de poder para repartir
ese poder, usando AGs.

In [26]:
import math
import random
import numpy as np


mejor_gen = 0
mejor_crom = []
mejor_aptitud = 0
mejor_pa = []

def generar_curules(Num_curules, Num_partidos): ##esta funcion devuelve el numero de curules que tiene cada partido
  curules = [random.randint(1, Num_partidos) for _ in range(Num_curules)]
  poder_partidos = []
  for i in range (1,Num_partidos+1):
    poder_partidos.append(curules.count(i))
  return poder_partidos

def generar_entidades(Num_entidades): ##esta funcion devuelve el peso politico de cada entidad
  entidades = []
  for i in range (1,Num_entidades+1):
    entidades.append(random.randint(1, 100))
  return entidades

##El cromosoma se define como una lista de L = numero de entidades que indica a que partido pertenece cada identidad

##decodifica un cromosoma
def decodifica(crom, curules, peso_politico):
    proporcion_curul = [c / sum(curules) for c in curules]
    poder_partidos  = [0 for _ in range(len(curules))]
    for i in range (1,len(crom)):
      poder_partidos[int(crom[i]- 1)] += peso_politico[i]
    poder_partidos = [c/sum(poder_partidos) for c in poder_partidos]
    return proporcion_curul, poder_partidos


def evalua(crom, curules, peso_politico): ## la funcion de aptitud es la diferencia entre lo esperado y lo asignado
    poder_esperado, poder_asignado = decodifica(crom, curules, peso_politico)
    ##funcion de aptitud
    y = 100*(1 - sum(abs(e - a) for e, a in zip(poder_esperado, poder_asignado))/(len(curules)*np.mean(poder_esperado)))
    if y < 0:
      y = 0
    return y, poder_esperado, poder_asignado

def generar_cromosoma(num_entidades, num_partidos):
    return [random.randint(1, num_partidos) for _ in range(num_entidades)]## cromosomas aleatorios

In [27]:
def next_generation (poblacion,N, curules, peso_politico):
  L = np.size(poblacion[0])
  prob_mutacion = 0.1/100
  aptitud = [evalua(x,curules, peso_politico)[0] for x in poblacion]
  aptitud = [x/sum(aptitud) for x in aptitud]
  new_generation = []
  ##ruleta aleatoria _  cruce
  while len(new_generation) <= N:
    for i in range (N):
      if random.random() <= aptitud[i] and len(new_generation) <= N:
        new_generation.append(poblacion[i])

  ##Cruce
  sons_generation = []
  for i in range(N//2):
    swipe = int(random.random()*L)
    son_1 = np.concatenate((new_generation[2*i][:swipe], new_generation[2*i + 1][swipe:]))
    son_2 = np.concatenate((new_generation[2*i + 1][:swipe], new_generation[2*i][swipe:]))
    sons_generation.append(son_1)
    sons_generation.append(son_2)

  ##Mutacion
  for i in range (N):
    for j in range (L):
      if random.random() < prob_mutacion:
        sons_generation[i][j] = int(random.randint(1, len(curules)))

  return sons_generation


In [28]:
def search_best_crom(poblacion, Gen, curules, peso_politico):
  global mejor_crom
  global mejor_gen
  global mejor_aptitud

  global mejor_pa

  for crom in poblacion:
    Aptitud, poder_esperado, poder_asignado = evalua(crom, curules, peso_politico)
    if Aptitud >= mejor_aptitud:
      mejor_crom = crom
      mejor_aptitud = Aptitud
      mejor_gen = Gen+1
      mejor_pa = poder_asignado



  print(f"\nGeneracion actual:{Gen+1}")
  print(f"=== Mejor cromosoma total: de la generacion {mejor_gen} ===")
  print("Poder esperado = ", poder_esperado)
  print("Poder asignado = ", mejor_pa)
  print(f"Precision = {mejor_aptitud}%")

In [29]:
def Ag(Num_curules = 50,Num_partidos =5, Num_entidades =  50,N = 100, Gen = 1000):

  Curules = generar_curules(Num_curules, Num_partidos)
  peso_politico = generar_entidades(Num_entidades)

  poblacion = [generar_cromosoma(Num_entidades, Num_partidos) for _ in range(N)] ##diseñamos la poblacion original

  for generacion in range (Gen):
    search_best_crom(poblacion,generacion,Curules, peso_politico)
    poblacion = next_generation(poblacion,N,Curules,peso_politico)

  print('\n\n')
  print('\n\n')
  print('\n\n Resultado final')
  #Imprimimos la tabla de poderes final de la mejor generacion
  print("Entidad | Poder | Partido al que pertenece")
  for i in range (Num_entidades):
    print('Entidad ', i+1,' | ',peso_politico[i],' | ',mejor_crom[i])
  print('\n\n')

  aptitud, poder_esperado, poder_asignado = evalua(mejor_crom,Curules, peso_politico)
  poder_asignado = poder_asignado*sum(peso_politico)

  print('Partido | Poder politico | Proporcion Poder | Num Curules | Proporcion Curules')
  for i in range (Num_partidos):
    print('Partido ', i+1,' | ',poder_asignado[i]*sum(peso_politico),' | ',poder_asignado[i]*100,'%  | ',Curules[i],' | ',poder_esperado[i]*100 ,'%   ')

  print('\n\n Precicion total: ',aptitud,'%')
Ag()


Se han truncado las últimas 5000 líneas del flujo de salida.
Poder asignado =  [0.20308747855917667, 0.22195540308747855, 0.18181818181818182, 0.21989708404802744, 0.1732418524871355]
Precision = 98.62778730703259%

Generacion actual:180
=== Mejor cromosoma total: de la generacion 105 ===
Poder esperado =  [0.2, 0.22, 0.18, 0.22, 0.18]
Poder asignado =  [0.20308747855917667, 0.22195540308747855, 0.18181818181818182, 0.21989708404802744, 0.1732418524871355]
Precision = 98.62778730703259%

Generacion actual:181
=== Mejor cromosoma total: de la generacion 105 ===
Poder esperado =  [0.2, 0.22, 0.18, 0.22, 0.18]
Poder asignado =  [0.20308747855917667, 0.22195540308747855, 0.18181818181818182, 0.21989708404802744, 0.1732418524871355]
Precision = 98.62778730703259%

Generacion actual:182
=== Mejor cromosoma total: de la generacion 105 ===
Poder esperado =  [0.2, 0.22, 0.18, 0.22, 0.18]
Poder asignado =  [0.20308747855917667, 0.22195540308747855, 0.18181818181818182, 0.21989708404802744, 0.173

Ejecutando el algoritmo genetico con 1000 generaciones y una poblacion de 100 cromosomas. Obtenemos una distribucion de los poderes muy uniforme con una precision de al menos 99% en la mayoria de los casos. Por lo cual podemos decir que este aloritmo soluciona el problema planteado mediante su comportamiento genetico.